In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [2]:
import os

# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax
import jax.numpy as jnp
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [3]:
jax.devices()

[CudaDevice(id=0)]

In [4]:
jax.devices('cpu')

[CpuDevice(id=0)]

In [5]:
o = jnp.ones(100)
o.device

CudaDevice(id=0)

In [6]:
from ernestogym.envs.single_agent.utils import parameter_generator

In [7]:
import os
print(os.getcwd())

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [8]:
pack_options = "ernestogym/ernesto_jax/data/battery/pack.yaml"
ecm = "ernestogym/ernesto_jax/data/battery/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery/models/aging/bolun_pack_dropflow.yaml"
world = "ernestogym/envs/single_agent/world_deg.yaml"
# world = "ernestogym/envs/single_agent/world_fading.yaml"

params = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    use_reward_normalization=True

)

params

{'battery': {'sign_convention': 'passive',
  'params': {'nominal_voltage': 350.4,
   'nominal_capacity': 60.0,
   'nominal_dod': 0.8,
   'nominal_lifetime': 3000,
   'v_max': 398.4,
   'v_min': 288.0,
   'temp_ambient': 298.15,
   'nominal_cost': 3000},
  'bounds': {'voltage': {'low': 288.0, 'high': 398.4},
   'current': {'low': -180.0, 'high': 60.0},
   'power': {'low': -71712.0, 'high': 23904.0},
   'temperature': {'low': 273.15, 'high': 323.15},
   'temp_ambient': {'low': 273.15, 'high': 313.15},
   'soc': {'low': 0.2, 'high': 0.8},
   'soh': {'low': 0.8, 'high': 1.0}},
  'init': {'voltage': 288.0,
   'current': 0.0,
   'power': 0.0,
   'temperature': 293.15,
   'temp_ambient': 293.15,
   'soc': 0.5,
   'soh': 1.0}},
 'input_var': 'current',
 'models_config': [{'type': 'electrical',
   'class_name': 'TheveninModel',
   'use_fading': False,
   'components': {'r0': {'selected_type': 'scalar',
     'scalar': 10.0,
     'lookup': {'table': 'r0_pack.csv',
      'inputs': [{'var': 'temper

In [9]:
from ernestogym.ernesto_jax.energy_storage.bess_degrading_dropflow import BatteryEnergyStorageSystem

In [10]:
state = BatteryEnergyStorageSystem.get_init_state(models_config=params['models_config'],
                                                  battery_options=params['battery'],
                                                  input_var=params['input_var'])
init_state = state
print(state)

BessBolunDropflowState(nominal_capacity=Array(60., dtype=float32, weak_type=True), nominal_cost=Array(3000, dtype=int32, weak_type=True), nominal_voltage=Array(350.4, dtype=float32, weak_type=True), nominal_dod=Array(0.8, dtype=float32, weak_type=True), nominal_lifetime=Array(3000, dtype=int32, weak_type=True), c_max=Array(60., dtype=float32, weak_type=True), temp_ambient=Array(293.15, dtype=float32, weak_type=True), v_max=Array(398.4, dtype=float32, weak_type=True), v_min=Array(288., dtype=float32, weak_type=True), elapsed_time=Array(0., dtype=float32, weak_type=True), electrical_state=ElectricalModelState(r0_nominal=Array(10., dtype=float32, weak_type=True), r0=Array(10., dtype=float32, weak_type=True), rc=RCState(resistance_nominal=Array(10., dtype=float32, weak_type=True), resistance=Array(10., dtype=float32, weak_type=True), capacity=Array(10., dtype=float32, weak_type=True), i_resistance=Array(0., dtype=float32, weak_type=True)), ocv_potential=Array(10., dtype=float32, weak_type=

In [11]:
# jax.tree.map(lambda a: jnp.array(a), state)

In [12]:
struct_original_state = jax.tree.map(lambda val: val.dtype, state)
struct_original_state

BessBolunDropflowState(nominal_capacity=dtype('float32'), nominal_cost=dtype('int32'), nominal_voltage=dtype('float32'), nominal_dod=dtype('float32'), nominal_lifetime=dtype('int32'), c_max=dtype('float32'), temp_ambient=dtype('float32'), v_max=dtype('float32'), v_min=dtype('float32'), elapsed_time=dtype('float32'), electrical_state=ElectricalModelState(r0_nominal=dtype('float32'), r0=dtype('float32'), rc=RCState(resistance_nominal=dtype('float32'), resistance=dtype('float32'), capacity=dtype('float32'), i_resistance=dtype('float32')), ocv_potential=dtype('float32'), is_active=dtype('bool'), v=dtype('float32'), i=dtype('float32'), p=dtype('float32'), v_rc=dtype('float32')), thermal_state=ThermalModelState(c_term=dtype('int32'), r_cond=dtype('float32'), r_conv=dtype('float32'), dv_dT=dtype('float32'), temp=dtype('float32'), heat=dtype('float32')), soc_state=SOCModelState(soc=dtype('float32'), soc_max=dtype('float32'), soc_min=dtype('float32')), soh=dtype('float32'), aging_state=BolunDro

In [15]:
from time import time
import numpy as np

a = np.random.uniform(-100, 100)

t1 = time()
state = BatteryEnergyStorageSystem.step(state, 25., 1.)
print(a)
print(time() - t1)
# print(state)

-98.15139586219527
0.0014340877532958984


In [16]:
jax.tree.map(lambda val: val.dtype, state) == struct_original_state

True

In [17]:
print(jax.tree.map(lambda val: val.dtype, state))
print(struct_original_state)

BessBolunDropflowState(nominal_capacity=dtype('float32'), nominal_cost=dtype('int32'), nominal_voltage=dtype('float32'), nominal_dod=dtype('float32'), nominal_lifetime=dtype('int32'), c_max=dtype('float32'), temp_ambient=dtype('float32'), v_max=dtype('float32'), v_min=dtype('float32'), elapsed_time=dtype('float32'), electrical_state=ElectricalModelState(r0_nominal=dtype('float32'), r0=dtype('float32'), rc=RCState(resistance_nominal=dtype('float32'), resistance=dtype('float32'), capacity=dtype('float32'), i_resistance=dtype('float32')), ocv_potential=dtype('float32'), is_active=dtype('bool'), v=dtype('float32'), i=dtype('float32'), p=dtype('float32'), v_rc=dtype('float32')), thermal_state=ThermalModelState(c_term=dtype('int32'), r_cond=dtype('float32'), r_conv=dtype('float32'), dv_dT=dtype('float32'), temp=dtype('float32'), heat=dtype('float32')), soc_state=SOCModelState(soc=dtype('float32'), soc_max=dtype('float32'), soc_min=dtype('float32')), soh=dtype('float32'), aging_state=BolunDro

In [21]:
t1 = time()
for i in range(10000):
    state = BatteryEnergyStorageSystem.step(state, 0.5, 1.)

print(time() - t1)

3.961975574493408


In [22]:
# jax.make_jaxpr(BatteryEnergyStorageSystem.step)(state, 0.5, 1.)

In [23]:
state

BessBolunDropflowState(nominal_capacity=Array(60., dtype=float32, weak_type=True), nominal_cost=Array(3000, dtype=int32, weak_type=True), nominal_voltage=Array(350.4, dtype=float32, weak_type=True), nominal_dod=Array(0.8, dtype=float32, weak_type=True), nominal_lifetime=Array(3000, dtype=int32, weak_type=True), c_max=Array(59.993687, dtype=float32, weak_type=True), temp_ambient=Array(293.15, dtype=float32, weak_type=True), v_max=Array(398.4, dtype=float32, weak_type=True), v_min=Array(288., dtype=float32, weak_type=True), elapsed_time=Array(40003., dtype=float32, weak_type=True), electrical_state=ElectricalModelState(r0_nominal=Array(10., dtype=float32, weak_type=True), r0=Array(10., dtype=float32, weak_type=True), rc=RCState(resistance_nominal=Array(10., dtype=float32, weak_type=True), resistance=Array(10., dtype=float32, weak_type=True), capacity=Array(10., dtype=float32, weak_type=True), i_resistance=Array(-0.50000435, dtype=float32, weak_type=True)), ocv_potential=Array(10., dtype=

# Test parallel of states

In [24]:
import jax
import jax.numpy as jnp

In [25]:
state = BatteryEnergyStorageSystem.get_init_state(models_config=params['models_config'],
                                                  battery_options=params['battery'],
                                                  input_var=params['input_var'])

state_list = []

n_states = 50

for i in range(n_states):
    state = BatteryEnergyStorageSystem.step(state, 0.5, 1.)
    state_list.append(state)

states = jax.tree.map(lambda *vals: jnp.array(vals), *state_list)

In [26]:
states.nominal_capacity.shape

(50,)

In [27]:
# batched_step = jax.vmap(BatteryEnergyStorageSystem.step)
batched_step = jax.jit(jax.vmap(BatteryEnergyStorageSystem.step))   #, backend='cpu')

In [31]:
t1 = time()
states = batched_step(states, jnp.ones(n_states) * 0.5, jnp.ones(n_states))
print(time() - t1)

0.0023374557495117188


In [32]:
# with jax.profiler.trace("/tmp/jax-trace", create_perfetto_link=True):
#     for i in range(1000):
#         states = batched_step(states, jnp.ones(n_states) * 0.5, jnp.ones(n_states))
#     jax.block_until_ready(states)

In [34]:
t1 = time()
for i in range(10000):
    states = batched_step(states, jnp.ones(n_states) * 0.5, jnp.ones(n_states))

jax.block_until_ready(states)
print(time() - t1)

11.574926137924194


In [35]:
states.nominal_capacity.device

CudaDevice(id=0)

## Jitted loop

In [36]:
from functools import partial

# @partial(jax.jit, static_argnums=[0])
# def maybe_loding(fun, states, act, times):
#     for i in range(500):
#         states = batched_step(states, act, times)
#     return states

@partial(jax.jit, static_argnums=[0, 1])
def fused_loop(fun, num, states, act, times):
    def body_fun(i, states):
        return fun(states, act[i], times[i])
    return jax.lax.fori_loop(0, num, body_fun, states)

### single battery

In [45]:
t1 = time()

num = 10000

actions = np.random.uniform(-100, 100, size=(num,))

state_new = fused_loop(BatteryEnergyStorageSystem.step, num, init_state, actions, jnp.ones(num)*60)

jax.block_until_ready(state_new)
print(time() - t1)

0.8235073089599609


### multiple batteries

In [48]:
t1 = time()

num = 10000

states_new = fused_loop(batched_step, num, states, jnp.ones((num, n_states)) * 0.5, jnp.ones((num, n_states)))

jax.block_until_ready(states_new)
print(time() - t1)

6.847105026245117
